In [1]:
import cv2
import numpy as np
import os
import glob
import scipy
import peakutils # Buscar implementación en C++
import seaborn as sns # Para graficar boxplots
from scipy.signal import savgol_filter # Buscar implementación en C++
from numpy.linalg import inv

Clases de archivos de test:
* Clasificados correctamente (aparentemente): {3,5,10,18,19,24}
* Clasificadas casi bien: {9,11,12,15,16,20,22,25}
* Archivos mal clasificados: {1,2,4,6,7,8,13,14,17,21,23,26,27,28,29,30,31,32} <br>
Algunas de las imágenes mal clasificadas se deben descartar.<br>
Para la próxima hay que guardar la información de ángulo de toma, ancho de cultivo y altura. <br>
<br>
Problemas detectados:
* El entre surco ocupa todo (1,2,4,7,8,14,17,20,21,23,27)
* El surco se corta
* El surco comienza pero termina enseguida
* El ancho de surco parece no ser correcto (9,11,12,16,20,22,25) <br>
Este problema es debido a que el ancho superior no se calcula sino que se pone de manera estática, hay que probar con otros valores
* Las rectas van para cualquier lado (1,6,13,26,30,31,32)

## Funciones varias de soporte

In [2]:
def mostrar_imgs(lbls, vec_img, reduction_ratio=0):
    if reduction_ratio >= 1:
        reduction_ratio = 0
    for i in range(len(vec_img)):
        h = vec_img[i].shape[0]
        h = (int)(h - h*reduction_ratio)
        w = vec_img[i].shape[1]
        w = (int)(w - w*reduction_ratio)
        img_rs = cv2.resize(vec_img[i], (w, h))
        cv2.imshow(lbls[i], img_rs)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def mostrar_img(lbl, img, reduction_ratio=0):
    if reduction_ratio >= 1:
        reduction_ratio = 0
    h = img.shape[0]
    h = (int)(h - h*reduction_ratio)
    w = img.shape[1]
    w = (int)(w - w*reduction_ratio)
    img_rs = cv2.resize(img, (w, h))
    cv2.imshow(lbl, img_rs)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

#-------------------------------------------------------------------------#    

class Line:
    def __init__(self, m, b, white_amount):
        self.m = m
        self.b = b
        self.white_amount = white_amount

    def __str__(self):
        return 'rho:' + str(self.rho) + ' theta: ' + str(self.theta) + 'white amount: ' + str(self.white_amount)

class MicroROI:
    def __init__(self, microimg, x_left, y_top):
        xc_sum = 0
        yc_sum = 0
        _sum = 0
        xc = int(microimg.shape[1] / 2)
        yc = int(microimg.shape[0] / 2)
        for x in range(0, microimg.shape[1]):
            for y in range(0, microimg.shape[0]):
                if (microimg[y, x] == 255):  # Cambiar a binario
                    xc_sum = xc_sum + x
                    yc_sum = yc_sum + y
                    _sum = _sum + 1
        if (_sum != 0):
            xc = int(xc_sum / _sum)
            yc = int(yc_sum / _sum)
        self.xc, self.yc = xc + x_left, yc + y_top

    def get_centroid(self):
        return self.xc, self.yc

    
def get_vegetation_in_line(img, m, b):
    sum = 0
    for i in range(0, img.shape[0]):
        if (img[i, int((i-b)/m)] == 255):
            sum = sum + 1
    return sum

def get_vegetation_amount(binary_img):
    white_pxls = np.sum(binary_img == 255)
    total = binary_img.shape[0] * binary_img.shape[1]
    return white_pxls / total

def get_maximum_points(horizontal_segment):
    histogram = row_histogram(horizontal_segment)
    smoothed_2dg = (savgol_filter(histogram, window_length=31,
                                  polyorder=1)).astype(np.int16)
    return peakutils.indexes(smoothed_2dg, thres=0.05/max(smoothed_2dg), min_dist=100)

def centroid(img):
    xc_sum = 0
    yc_sum = 0
    _sum = 0
    xc = int(img.shape[1] / 2)
    yc = int(img.shape[0] / 2)
    for x in range(0, img.shape[1]):
        for y in range(0, img.shape[0]):
            if (img[y, x] == 255):  # Cambiar a binario
                xc_sum = xc_sum + x
                yc_sum = yc_sum + y
                _sum = _sum + 1
    if (_sum != 0):
        xc = int(xc_sum / _sum)
        yc = int(yc_sum / _sum)
    return xc, yc

def get_mean_distance(indexes):
    mean = 0
    for i in range(1, indexes.shape[0]):
        mean = mean + indexes[i] - indexes[i-1]
    return mean / indexes.shape[0]


def is_crop_or_furrow_lineal(coef, p0, crop_width, furrow_width):
    # Crop: 1, Crop furrow: -1, None: 0
    x, y = p0
    crop_width = int(crop_width / 2)
    # El último termno es para cerrar el surco (ya que es promedio y no el mayor, hay que cambiar al mayor)
    furrow_width = int(furrow_width)
    for i_line in range(0, coef.shape[0]):  # Paso por cada curva
        # (y, m, b):
        current_x = get_x_in_line(y, coef[i_line, 1], coef[i_line, 0])
        if (x >= current_x - crop_width and x <= current_x + crop_width):
            return 1
        if (x >= current_x - furrow_width and x <= current_x + furrow_width):
            return -1
    return 0

def get_x_in_line(y, m, b):
    return -1 if (m == 0) else int((y-b)/m)

# ------------------------------------------------#
# Least absolute deviation regression
def least_squares_line(x_list, y_list, point0):
    x_list = np.array(x_list, np.uint64)
    y_list = np.array(y_list, np.uint64)
    scalar_factor = 100
    x_list = np.true_divide(x_list, scalar_factor)
    y_list = np.true_divide(y_list, scalar_factor)
    sum_x = np.sum(x_list)
    sum_y = np.sum(y_list)
    sum_xy = np.sum(np.multiply(x_list, y_list))
    sum_x_squared = np.sum(np.multiply(x_list, x_list))
    numerator = x_list.shape[0] * sum_xy - (sum_x * sum_y)
    denominator = x_list.shape[0] * sum_x_squared - (sum_x * sum_x)
    # Si 0 es denominador, es una recta vertical, la pendiente es infinito
    m = numerator / denominator if denominator != 0 else 9999
    x_p0, y_p0 = point0
    b = int(y_p0 - m * x_p0)
    return m, b  # Coeficientes: Lineal y independiente

def mean(values_list):
    values_list = np.array(values_list, np.uint64)
    return (np.sum(values_list) / values_list.shape[0])

def img_to_color_index(img, index):
    if (index == 'cive'):
        return get_CIVE(img)
    elif (index == 'exg'):
        return get_ExG(img)
    else:
        return img

# r'=r/(r+g+b) g'=g/(r+g+b) b'=b/(r+g+b)
def get_normalized(img):
    img = img.astype(np.uint16)
    denominator = img[:, :, 0] + img[:, :, 1] + img[:, :, 2]
    # Max 255+255+255, Min 1
    denominator = np.where(denominator == 0, 1, denominator)
    return cv2.merge((img[:, :, 0]/denominator, img[:, :, 1]/denominator, img[:, :, 2]/denominator))

def get_CIVE(img):
    img_norm = get_normalized(img)
    return 0.441*img_norm[:, :, 0]-0.811*img_norm[:, :, 1]+0.385*img_norm[:, :, 2]+18.78745

def get_ExG(img):
    r, g, b = cv2.split(img)
    r_mx = np.amax(r)
    g_mx = np.amax(g)
    b_mx = np.amax(b)
    if (r_mx == 0):
        r_mx = 1
    if (g_mx == 0):
        g_mx = 1
    if (b_mx == 0):
        b_mx = 1
    r = r / r_mx
    g = g / g_mx
    b = b / b_mx
    denominator = r + g + b
    denominator = np.where(denominator == 0, 1, denominator)
    r = r / denominator
    g = g / denominator
    b = b / denominator
    return 2*g-r-b

def row_histogram(img_in):
    vector = np.zeros(img_in.shape[1], np.uint16)
    for col in range(0, img_in.shape[1]-1):
        vector[col] = np.sum(img_in[:, col] == 255)
    return vector

def v_crop_top(img, crop_ratio):
    if (crop_ratio < 0 or crop_ratio > 0.9):
        crop_ratio = 0
    h2 = img.shape[0]
    h1 = (int)(h2*crop_ratio)
    w = img.shape[1]
    return img[h1:h2, 0:w]

##### Cambios
* Corroborar que todas las variables se utilizan, declaran, etc.
* Verificar que es mejor, least absolute deviation o least squares
* Hacer la clase punto para la lista de coordenadas de los centroides
* Conseguir los algortimos que utilizan librerías, buscar soluciones sencillas con mismos resultados

## Variables de configuración

In [3]:
# Reducción del tamaño original para poder ver las imágenes en tamañas chicas
reduction_to_see = 0.5

# Kernels para utilizar
kernel = np.ones((3, 3), np.uint8)
kernel_verticales = np.array([[-2, 4, -2],  # Segmentos verticales
                              [-2, 4, -2],
                              [-2, 4, -2]])
kernel_vertical = np.array([
    [0, 1, 0],
    [0, 1, 0],
    [0, 1, 0]
], np.uint8)
kernel_diagonal_1 = np.array([
    [0, 0, 1],
    [0, 1, 0],
    [1, 0, 0]
], np.uint8)
kernel_diagonal_2 = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
], np.uint8)
kernel_horizontal = np.array([
    [0, 0, 0],
    [1, 1, 1],
    [0, 0, 0]
], np.uint8)

debugging = False


                                        #-------DEPENDIENTES-------#
#-------Depende del ángulo de cámara-------#
# Cuanto se corta de la imagen para quitar el punto de fuga
porcentaje_de_crop = 0.1

#-------Depende del tipo de planta y su estadio-------#
# Ancho de línea de cultivo en píxeles
crop_width_bottom = 90
crop_width_top = 20
# Este porcentaje de ancho es dependiente del tipo de cultivo y el estadío en el que se encuentra
addition_percentage = 0.10
addition = crop_width_bottom * addition_percentage

#-------Depende de la luminosidad-------#
# Índices posibles: cive exg (cive para nublado, exg para soleado)
indice_seleccionado = 'exg'


                                        #-------VERIFICABLES-------#
# Para Hough
# Rho: Resolución de distancia para el acumulador de píxeles
in_rho = 18
# Theta: Resolución de ángulos para el acumulador de radianes
in_theta = np.pi/90
# Threshold: Parametro de threshold para el acumulador, pasan las líneas con mayor cantidad de votos
in_threshold = 500
# Ángulos de control configurables 
ang_min_tercio_1 = 0.2
ang_max_tercio_1 = 0.49
ang_min_tercio_2 = 2.65
ang_max_tercio_2 = 2.92
ang_min_tercio_3 = 2.75
ang_max_tercio_3 = 0.39


                                        #-------AJUSTABLES-------#
# Un cuarto de la imagen original
rs_ratio = 0.25
# Porcentaje mínimo requerido para aplicar erosión y resaltar los surcos
porcentaje_para_erosion = 0.15
#Porcentaje de maleza mínimo que tiene que tener un cuadrante para detectarlo con maleza
weed_porc = 0.08

## Algoritmo
### Selección de imagen

In [4]:
# Elección de imagen de prueba
# Cambiar algoritmo a una función llamable para iterar sobre varias imágenes
# Test 1: Recorte de 0.2
# Test 2: No hacerle recorte
# path_to_imgs = 'img/test/'
# file_name = 'test1'
# _original = cv2.imread(path_to_imgs + file_name + '.png', cv2.IMREAD_COLOR)
# if _original is None:
#     print('Warning: imagen no cargada')

### Detección

In [5]:
# Elección de imagen de prueba
# Cambiar algoritmo a una función llamable para iterar sobre varias imágenes
# Test 1: Recorte de 0.2
# Test 2: No hacerle recorte
for i in range(1,33):
    path_to_imgs = 'img/test/'
    file_name = 'test' + str(i)
    print(file_name)
    _original = cv2.imread(path_to_imgs + file_name + '.png', cv2.IMREAD_COLOR)
    if _original is None:
        print('Warning: imagen no cargada')
    else:

        #-------PREPARACIÓN DE DATOS-------#

        # Cortar la parte superior de la imagen (esto se calcula de manera manual, depende del ángulo de pitch que tenga la cámara)
        # Módulo IP de control que pérmite el paso de los píxeles por debajo
        original = v_crop_top(_original, porcentaje_de_crop)

        #------------------------#
        if (debugging):
            mostrar_img('cropped', original, reduction_to_see)
        #------------------------#

        # Obtención de imagen con índice
        # Para módulo IP - Debe devolver índice en 8 bits
        # Se utilizan los índice CIVE y ExG (Siendo el índice CIVE eficiente para días parcial y totalmente nublados, y el índice ExG para días soleados)
        img_veg = img_to_color_index(original, indice_seleccionado)

        # La convierto en binario de 8 bits para aplicar Otsu (No debe ser parte del VHDL, ya debe devolver 8 bits)
        amax = np.amax(img_veg)
        amin = np.amin(img_veg)
        scalingIndex = 255/(amax-amin)
        img_veg = ((img_veg-amin)*scalingIndex)
        img_veg = img_veg.astype(np.uint8)
        # La parte de inversión de valor debe estar dentro del calculo del índice
        if (indice_seleccionado == 'exg'):
            img_veg = cv2.bitwise_not(img_veg)

        # Otsu (Comienza parte pesada de algoritmo, a partir de acá debe ir en un mismo módulo IP)
        ret, img_veg = cv2.threshold(
            img_veg, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

        # Apertura para eliminar ruido
        img_veg = cv2.morphologyEx(
            img_veg, cv2.MORPH_OPEN, kernel, iterations=1)

        # Utilizar el algoritmo con un reescalado de la imagen (Se debe poder configurar el tamaño de reducción y el algortimo utilizado)
        img_veg = cv2.resize(img_veg, None, fx=rs_ratio,
                             fy=rs_ratio, interpolation=cv2.INTER_AREA)
        new_height = img_veg.shape[0]
        new_width = img_veg.shape[1]

        # Utilizo eroción para acentuar la diferencia entre líneas de cultivo, cuando la cantidad de vegetación es alta
        v_a = get_vegetation_amount(img_veg)
        if (v_a > porcentaje_para_erosion):
            img_veg_eroded = (cv2.erode(img_veg, kernel))
        else:
            img_veg_eroded = img_veg

        #------------------------#
        if (debugging):
            mostrar_imgs(['img_veg', 'img_veg_eroded'], [img_veg, img_veg_eroded])
        #------------------------#

        #-------HISTOGRAMA-------#
        # Calcular histograma de la franja inferior de la imágen, detectar la cantidad de picos según la cantidad de lineas se busquen, si hay más o menos de lo buscado se debe descartar la imagen.
        h_space = int(new_height/12)
        indexes = get_maximum_points(img_veg[new_height -
                                             h_space:new_height, 0:new_width])

        #-------TRANSFORMADA DE HOUGH-------#
        middle = new_height-int(new_height/2)
        bottom_image = img_veg_eroded[middle:new_height,
                                      0:new_width]
        # Calculo líneas de la TH (requiere edge detection previo, ver cual ocupa menos de los métodos)
        edges = cv2.filter2D(bottom_image, -1, kernel_verticales)
        lines = cv2.HoughLines(edges, in_rho, in_theta, in_threshold)

        # Control de las líneas detectadas
        third_1 = (int)(bottom_image.shape[1]/3)
        third_2 = (int)(bottom_image.shape[1]*(2/3))
        epsilon = (int)(0.01*bottom_image.shape[1])
        delta = int(crop_width_bottom/2)  # Medio ancho de surco para control
        upper_limit = indexes + delta
        lower_limit = indexes - delta
        # La cantidad de grupos es dinámico, en c++ tendrá que ser estático con un tope, cada elemento del grupo tiene: Rho, Theta y cantidad de vegetación (la clase Line)
        groups = {}
        if type(lines) is np.ndarray:
            for line in lines:
                for rho, theta in line:
                    if (np.sin(theta) != 0):
                        m = -(np.cos(theta)/np.sin(theta))
                        b = rho/np.sin(theta)
                        x = int((bottom_image.shape[0]-b)/m)
                        group = -1
                        for i in range(0, len(indexes)):
                            if (x <= upper_limit[i] and x >= lower_limit[i]):
                                group = i
                        # Ángulos de control configurables
                        correct_angle_and_position = (x < third_1 and theta < ang_max_tercio_1 and theta > ang_min_tercio_1) \
                            or (x > third_2 and theta > ang_min_tercio_2 and theta < ang_max_tercio_2) \
                            or (x > third_1 and x < third_2 and (theta > ang_min_tercio_3 or theta < ang_max_tercio_3))
                        if (x > 0 and x < bottom_image.shape[1] and group != -1 and correct_angle_and_position):
                            v_a = get_vegetation_in_line(bottom_image, m, b)
                            if group not in groups.keys():  # Es necesario algún control en la estructura de C++ para lograr esto
                                groups[group] = Line(m, b, v_a)
                            else:
                                if (groups[group].white_amount < v_a):
                                    groups[group] = Line(m, b, v_a)

        #------------------------#
        if (debugging):
            with_lines = bottom_image.copy();
            with_lines = cv2.merge((with_lines,with_lines,with_lines))
            for key in groups.keys():
                line = groups.get(key)
                m = line.m
                b = line.b
                y0 = 0
                x0 = int((y0-b)/m)
                y1 = with_lines.shape[0]
                x1 = int((y1-b)/m)
                cv2.line(with_lines, (x0, y0), (x1, y1), (0, 255, 0), 2)
            mostrar_img('Hough', with_lines)
        #------------------------#

        #-------SEGMENTACIÓN-------#
        # Segmentación de la imagen en partes que disminuyen un 5% cuando se avanza hacia arriba
        # La segmentación es dependiente del alto de la imagen, ya que siempre se calculan 12, por tanto se puede realizar una vez cuando cambia el
        # valor de corte, de proporción de disminución y de cambio en resolución
        # Puede utilizarse un módulo IP aparte para que lo calcule, hay que calcular el alto de la en base a los parametros de la imagen
        total_y = new_height

        # Para obtener los 4 segmentos inferiores
        y_segments_bottom = np.arange(11, 15)
        y_segments_top = np.arange(0, 9)

        # Para obtener los 8 segmentos superiores
        y_segments_bottom = (np.multiply((np.true_divide(
            y_segments_bottom, np.sum(y_segments_bottom))), middle)).astype(np.uint16)
        y_segments_top = (np.multiply((np.true_divide(
            y_segments_top, np.sum(y_segments_top))), middle)).astype(np.uint16)
        y_segments_bottom = y_segments_bottom[::-1]
        y_segments_top = y_segments_top[::-1]
        y_segments_bottom[0] = y_segments_bottom[0] + \
            middle - np.sum(y_segments_bottom)
        y_segments_top[0] = y_segments_top[0] + middle - np.sum(y_segments_top)
        y_segments = np.concatenate((y_segments_bottom[0:len(
            y_segments_bottom) - 1], y_segments_top[0:len(y_segments_top)]))
        for i in range(len(y_segments) - 2, -1, -1):
            y_segments[i] = y_segments[i] + y_segments[i+1]
        y_segments = np.concatenate(([new_height], y_segments))


        #-------CÁLCULO DE MICROROIS-------#
        # Se calcula el centro del ROI en base a las rectas encontradas por TH
        # Si el microROI no tiene píxeles blancos se asume que el centro es el centroide

        # Matrices de puntos (puede ser una clase) depende de la cantidad de rectas (cantidad dinámica)
        x_centroids_matrix = np.zeros((len(indexes), len(y_segments) - 1), np.uint16)
        y_centroids_matrix = np.zeros((len(indexes), len(y_segments) - 1), np.uint16)
        crop_width = crop_width_bottom
        crop_width_decay_seg = (crop_width_bottom - crop_width_top) / \
            (len(y_segments) - 1)  # (90-30)/12
        # Ya que la recta está calculada para la parte inferior, es necesario un offset de b para transladarla
        img_offset = middle
        # Contador del segmento en el que se posiciona, los valores son: [1, ..., 12]
        segment_index = 1
        while y_segments[segment_index] >= middle:
            for key in groups.keys():
                # Obtener cada descriptor de línea
                line = groups.get(key)
                m = line.m
                b = line.b + img_offset
                y1, y2 = y_segments[segment_index -
                                    1], y_segments[segment_index]  # Inferior, superior
                # El x del inferior, respecto a la recta
                x_temp = get_x_in_line(y1, m, b)
                # Desplazamientos según ancho de línea
                x1, x2 = x_temp - int(crop_width/2), x_temp + int(crop_width/2)
                # Pasarle microimagen con desplazamiento en x e y
                centroid = MicroROI(img_veg_eroded[y2:y1, x1:x2], x1, y2)
                x_centroids_matrix[key, segment_index - 1], y_centroids_matrix[key,
                                                                               segment_index - 1] = centroid.get_centroid()
                key = key + 1  # Próxima línea
            # Nuevo ancho de microROI
            crop_width = crop_width - crop_width_decay_seg
            segment_index = segment_index + 1  # Próxima franja

        # Usar la recta con los últimos cuatro elementos para el ROI del siguiente nivel
        # Calcula centroides del siguiente nivel y agregarlos en las matrices correspondientes
        last_centroids_index = 0
        while segment_index != (len(y_segments)):
            for line_index in range(0, x_centroids_matrix.shape[0]):  # Para cada línea
                # Tomo elementos de [i, i+3] correspondientes a los últimos cuatro centroides
                x_list = x_centroids_matrix[line_index,
                                            last_centroids_index:last_centroids_index + 4]
                y_list = y_centroids_matrix[line_index,
                                            last_centroids_index:last_centroids_index + 4]
                last_centroid = (
                    x_centroids_matrix[line_index, last_centroids_index+3], y_centroids_matrix[line_index, last_centroids_index+3])

                m, b = least_squares_line(x_list, y_list, last_centroid)
                y1, y2 = y_segments[segment_index -
                                    1], y_segments[segment_index]  # Inferior, superior
                x1, x2 = get_x_in_line(
                    new_height, m, b), get_x_in_line(y2, m, b)
                x_temp = get_x_in_line(y1, m, b)
                # Desplazamientos según ancho de línea
                x1, x2 = x_temp - int(crop_width / 2), x_temp + \
                    int(crop_width / 2)
                # Pasarle microimagen con desplazamiento en x e y
                centroid = MicroROI(img_veg_eroded[y2:y1, x1:x2], x1, y2)
                x_centroids_matrix[line_index, segment_index-1], y_centroids_matrix[line_index,
                                                                                    segment_index-1] = centroid.get_centroid()
            segment_index = segment_index + 1  # Próxima franja
            crop_width = crop_width - crop_width_decay_seg
            last_centroids_index = last_centroids_index + 1


        #------MÉTODO DE MÍNIMOS CUADRADOS-------#
        # Una vez obtenidos todos los centroides de cada línea se realiza una análisis de regresión
        y1, y2 = 0, new_height
        y = np.arange(y1, y2)
        v_amount_line = 0

        # Independiente y líneal (La cantidad de columnas depende de la cantidad de surcos)
        lineal_coef = np.zeros([x_centroids_matrix.shape[0], 2],
                               np.float16)
        for line_index in range(0, x_centroids_matrix.shape[0]):  # Para cada línea
            x_list = x_centroids_matrix[line_index, 0:x_centroids_matrix.shape[1]]
            y_list = y_centroids_matrix[line_index, 0:y_centroids_matrix.shape[1]]

            # Mínimos cuadrados para recta
            x_mean = int(mean(x_list))
            y_mean = int(mean(y_list))
            lineal_coef[line_index, 1], lineal_coef[line_index,
                                                    0] = least_squares_line(x_list, y_list, (x_mean, y_mean))

        # Selección y verificación (reglas para verificar que sean rectas válidas)
        # 1) Regularidad en el espaciado entre líneas
        # 2) No intersección dentro del ROI ni extensión por debajo de la imagen

        # -------------------------Agregar control de rectas---------------------------------------


        #-------MASCARAS DE MALEZA-------#
        # Generación de máscaras con corrección según tipo de planta y estadío
        crop_mask = np.zeros(
            (new_height, new_width), np.uint8)
        furrow_crop_mask = crop_mask.copy()
        y1, y2 = 0, crop_mask.shape[0]

        crop_width = crop_width_bottom + addition
        crop_width_decay = (crop_width_bottom - crop_width_top) / y2

        # Media de espacio en la base
        furrow_width_bottom = int(get_mean_distance(np.sort(indexes)))
        furrow_width = furrow_width_bottom
        # Para calcular el espacio entre surcos en el tope
        indexes_top = np.zeros(indexes.shape)
        for i_line in range(0, len(lineal_coef)):
            indexes_top[i_line] = get_x_in_line(
                0, lineal_coef[i_line, 1], lineal_coef[i_line, 0])
        furrow_width_top = int(get_mean_distance(np.sort(indexes_top)))


        furrow_width_decay = (furrow_width_bottom - furrow_width_top) / y2
        for y in range(y2 - 1, y1 - 1, -1):  # Desde la base la parte superior de la imagen
            for x in range(0, crop_mask.shape[1]):
                result = is_crop_or_furrow_lineal(
                    lineal_coef, (x, y), crop_width, furrow_width)
                if (result == 1):
                    crop_mask[y, x] = 255
                if (result == -1):
                    furrow_crop_mask[y, x] = 255
            crop_width = crop_width - crop_width_decay
            furrow_width = furrow_width - furrow_width_decay

        # Para la versión reducida
        furrow_crop_mask_vegetation = cv2.bitwise_and(furrow_crop_mask, img_veg)

        # Disminución del contenido de vegetación en el entre surco para resaltar la maleza
        furrow_crop_mask_vegetation = cv2.morphologyEx(
            furrow_crop_mask_vegetation, cv2.MORPH_OPEN, kernel, iterations=1)

        # Se puede disminuir el costo verificando solo en la parte interna de la máscara de maleza, hace falta conseguir puntos que pertenecen a ella
        # Itero sobre cada franja
        for y_index in range(0, len(y_segments) - 1):  # Itera 12 veces
            # En cada franja separo en cuadrantes separados por líneas de cultivo (cantidad de líneas + 1)
            # En cada cuadrante le sumo la cantidad de vegetación y el tamaño (tamaño tendrá 1 más para no dividir por 0)
            weed_amount = np.zeros(lineal_coef.shape[0] + 1, np.uint16)
            total_pxls = np.ones(lineal_coef.shape[0] + 1, np.uint16)
            x_by_line = np.zeros(lineal_coef.shape[0], np.uint16)
            # Obtengo los x de cada línea en la base de la franja, para realizar la separación por cuadrantes
            for i in range(0, lineal_coef.shape[0]):  # Itera #L veces
                x_by_line[i] = get_x_in_line(
                    y_segments[y_index], lineal_coef[i, 1], lineal_coef[i, 0])
            # Itero en la franja correspondiente
            # Itera el alto de segmento (máx es 1/12 * (alto de imagen disminuida))
            for y in range(y_segments[y_index + 1], y_segments[y_index]):
                furrow_index = 0
                x = 0
                # Itera máx el ancho de la imagen
                while ((furrow_index < len(weed_amount)) and (x < crop_mask.shape[1])):
                    # Me saco todos los 0's del comienzo, si es que tiene
                    while ((x < crop_mask.shape[1]) and (furrow_crop_mask[y, x] == 0)):
                        x = x + 1
                    # Colóco el índice donde pertenece
                    while ((furrow_index < x_by_line.shape[0]) and (x >= x_by_line[furrow_index])):
                        furrow_index = furrow_index + 1
                    while ((x < crop_mask.shape[1]) and (furrow_crop_mask[y, x] == 255)):
                        # Procesamiento dentro de surco
                        total_pxls[furrow_index] = total_pxls[furrow_index] + 1
                        if furrow_crop_mask_vegetation[y, x] > 10:
                            weed_amount[furrow_index] = weed_amount[furrow_index] + 1
                        x = x + 1
                    furrow_index = furrow_index + 1

            # Adapto la máscara de surco acorde a si cada cuadrante pasa un porcentaje de maleza
            weed_amount = np.true_divide(weed_amount, total_pxls)
            for y in range(y_segments[y_index + 1], y_segments[y_index]):
                furrow_index = 0
                x = 0
                # Itera máx el ancho de la imagen
                while ((furrow_index < len(weed_amount)) and (x < crop_mask.shape[1])):
                    # Me saco todos los 0's del comienzo, si es que tiene
                    while ((x < crop_mask.shape[1]) and (furrow_crop_mask[y, x] == 0)):
                        x = x + 1
                    # Colóco el índice donde pertenece
                    while ((furrow_index < x_by_line.shape[0]) and (x >= x_by_line[furrow_index])):
                        furrow_index = furrow_index + 1
                    # Mientras que este en rango, y si pasa el umbral, se marca el sector con maleza
                    _data = 255 if (furrow_index < len(weed_amount)
                                    and weed_amount[furrow_index] > weed_porc) else 0
                    # Mientras que esté dentro de la imagen y sea parte del entre surco
                    while ((x < crop_mask.shape[1]) and (furrow_crop_mask[y, x] == 255)):
                        furrow_crop_mask[y, x] = _data
                        x = x + 1
                    furrow_index = furrow_index + 1

        # Clasificación sectorizada
        crop_mask = cv2.resize(crop_mask, None, fx=(
            1/rs_ratio), fy=(1/rs_ratio), interpolation=cv2.INTER_AREA)
        furrow_crop_mask = cv2.resize(furrow_crop_mask, None, fx=(
            1/rs_ratio), fy=(1/rs_ratio), interpolation=cv2.INTER_AREA)
        zeros = np.zeros(crop_mask.shape, np.uint8)
        mask = cv2.merge((zeros, crop_mask, furrow_crop_mask))
        total = cv2.add(original, mask)
        cv2.imwrite(path_to_imgs + file_name + '_'+ 'clasificado_sectores' +'.png', total)
        if (debugging):
            mostrar_imgs(['original', 'clasificada'], [original, total], reduction_to_see)

test1
test2
test3
test4
test5
test6
test7
test8
test9
test10
test11
test12
test13
test14
test15
test16
test17
test18
test19
test20
test21
test22
test23
test24
test25
test26
test27
test28
test29
test30
test31
test32
